# Visual overview of the geometric outputs produced by the pipeline

In [19]:
import numpy as np
import trimesh
import cv2
import plotly.graph_objects as go
from pyransac3d import Plane

In [ ]:
"""
1. Load input images from files into a list.
"""
# Load input images
images = []
N = 6  # Number of input images
for i in range(N):
    image = cv2.imread(f"geometry_{2*i+1}.jpeg") # Enter the path for the input images
    images.append(image)
input_height, input_width, _ = image.shape  # Get dimensions from one image

In [ ]:
"""
2. Load 3D scene geometry from a GLB file using trimesh.
"""
# Load a 3D mesh from a GLB file:
mesh = trimesh.load("scene.glb")

# Extract individual geometries from the loaded scene:
geometries = list(mesh.geometry.values())

In [22]:
"""
3. Extract 3D point clouds from the mesh and detect a ceiling plane using RANSAC.
"""
# Extract point clouds from the mesh
all_points = []
# Collect all points from the ceiling point cloud:
for name, geometry in mesh.geometry.items():
    if isinstance(geometry, trimesh.PointCloud):
        points = geometry.vertices
        all_points.append(points)
all_points = np.vstack(all_points)
# Fit the ceiling plane using RANSA
top_points = all_points
# RANSAC to fit the ceiling plane and get its equation:
plane_model = Plane()
plane_equation, inliers = plane_model.fit(top_points, thresh=0.00001)
plane_equation = np.array(plane_equation)
ceiling_points = top_points[inliers]

print("%d inliers" % (len(inliers)))
print("Plane equation :", plane_equation)

546 inliers
Plane equation : [0.95913923 0.22883252 0.16639598 0.67160107]


## Visualizing the found ceiling plane and the point cloud and adding the mesh representing the plane to the 3D reconstruction (file .glb)

In [24]:
trace_points = [] 
trace_meshes = []

# Colors used for different elements (point clouds, cameras, etc.)
colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta']

camera_id = 0
color_counter = 0

# Loop through each geometry in the mesh
# Used to visualize the ceiling point cloud and camera positions
for name, geometry in mesh.geometry.items():
    
    # If the geometry is a point cloud (e.g., ceiling)
    if isinstance(geometry, trimesh.PointCloud):
        # Limit to 1500 randomly selected points to avoid memory issues
        num_points = min(1500, len(geometry.vertices))
        sampled_points = np.random.choice(len(geometry.vertices), num_points, replace=False)
        points = geometry.vertices[sampled_points]
        
        # Add point cloud as a 3D scatter plot
        trace_points.append(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(color=colors[color_counter % len(colors)], size=2),
            name=f"PointCloud {name}"
        ))
        color_counter += 1

    # If the geometry is a mesh (used to represent cameras)
    elif isinstance(geometry, trimesh.Trimesh):
        for face in geometry.faces:
            vertices = geometry.vertices[face]
            # Only plot faces that are valid triangles (3 unique vertices)
            if len(np.unique(vertices, axis=0)) == 3:
                trace_meshes.append(go.Mesh3d(
                    x=vertices[:, 0],
                    y=vertices[:, 1],
                    z=vertices[:, 2],
                    color=colors[camera_id % len(colors)],
                    opacity=1,
                    name=f"Camera #{camera_id}"
                ))
        # Increment camera ID if it's a new camera (48 faces = one camera model)
        if len(geometry.faces) == 48: 
            camera_id += 1

# === Plotting the detected ceiling plane ===
xlim = [top_points[:, 0].min(), top_points[:, 0].max()]
ylim = [top_points[:, 1].min(), top_points[:, 1].max()]
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 10), np.linspace(ylim[0], ylim[1], 10))
a, b, c, d = plane_equation
zz = (-a * xx - b * yy - d) / c

# Add the ceiling plane as a translucent surface
trace_meshes.append(go.Surface(
    x=xx, y=yy, z=zz, opacity=0.2, colorscale='Oranges', showscale=False,
    name='Ceiling plane'
))

# === Normalize the scale of all 3 axes to avoid distortion ===
xlim = [points[:, 0].min(), points[:, 0].max()]
ylim = [points[:, 1].min(), points[:, 1].max()]
zlim = [points[:, 2].min(), points[:, 2].max()]
dx = xlim[1] - xlim[0]
dy = ylim[1] - ylim[0]
dz = zlim[1] - zlim[0]

# Set uniform axis range based on the longest dimension
if dx >= dy and dx >= dz:
    xmin = (xlim[0]+xlim[1])/2 - dx/2
    xmax = (xlim[0]+xlim[1])/2 + dx/2
    ymin = (ylim[0]+ylim[1])/2 - dx/2
    ymax = (ylim[0]+ylim[1])/2 + dx/2
    zmin = (zlim[0]+zlim[1])/2 - dx/2
    zmax = (zlim[0]+zlim[1])/2 + dx/2
elif dy >= dx and dy >= dz:
    xmin = (xlim[0]+xlim[1])/2 - dy/2
    xmax = (xlim[0]+xlim[1])/2 + dy/2
    ymin = (ylim[0]+ylim[1])/2 - dy/2
    ymax = (ylim[0]+ylim[1])/2 + dy/2
    zmin = (zlim[0]+zlim[1])/2 - dy/2
    zmax = (zlim[0]+zlim[1])/2 + dy/2
else:
    xmin = (xlim[0]+xlim[1])/2 - dz/2
    xmax = (xlim[0]+xlim[1])/2 + dz/2
    ymin = (ylim[0]+ylim[1])/2 - dz/2
    ymax = (ylim[0]+ylim[1])/2 + dz/2
    zmin = (zlim[0]+zlim[1])/2 - dz/2
    zmax = (zlim[0]+zlim[1])/2 + dz/2

# === Create the layout for 3D plot ===
layout = go.Layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(range=[xmin, xmax]),
        yaxis=dict(range=[ymin, ymax]),
        zaxis=dict(range=[zmin, zmax])
    ),
    title='Cameras, point cloud, and estimated ceiling plane',
    showlegend=True
)

# Show the figure combining point clouds and mesh elements
fig = go.Figure(data=trace_points + trace_meshes, layout=layout)
fig.show()

# === Create a mesh for the detected ceiling plane ===
vertices = np.column_stack((xx.ravel(), yy.ravel(), zz.ravel()))
faces = []
nx, ny = xx.shape
for i in range(nx - 1):
    for j in range(ny - 1):
        idx = lambda x, y: x * ny + y
        # Triangulate the grid for the surface mesh
        faces.append([idx(i, j), idx(i+1, j), idx(i, j+1)])
        faces.append([idx(i+1, j), idx(i+1, j+1), idx(i, j+1)])
plan_mesh = trimesh.Trimesh(vertices=vertices, faces=np.array(faces))

# === Create a new 3D scene including the original mesh and the new plane ===
new_scene = trimesh.Scene()
for name, geom in mesh.geometry.items():
    new_scene.add_geometry(geom, node_name=name)
new_scene.add_geometry(plan_mesh, node_name="plan_detecte")

# === Export the new scene to a GLB file ===
new_scene.export("C:/Users/diaea/Downloads/scene_12_plan_1.glb")
print("Export terminé vers scene_res1.glb")

Export terminé vers scene_res1.glb


----------------------------------------------------------------------------------

In [26]:
"""
4. For each camera:
   - Extract camera plane and center
   - Filter corners pointing toward the ceiling
   - Backproject valid image corners onto the ceiling plane
"""


# Helper function to compute intersection of line and plane
def plane_line_intersection(plane_equation, P1, P2):
    a, b, c, d = plane_equation
    # The coordinates of the two points defining the line:
    x1, y1, z1 = P1
    x2, y2, z2 = P2
    # Compute the denominator of the parametric intersection formula:
    denominator = a * (x2 - x1) + b * (y2 - y1) + c * (z2 - z1)
    # Check if the line is parallel to the plane:
    if np.abs(denominator) < 1e-6:
        print("The line is parallel to the plane (no intersection).")
        return None
    # Compute the parameter t for the point of intersection:
    t = -(a * x1 + b * y1 + c * z1 + d) / denominator
    # Calculate the intersection point:
    intersection = (x1 + t * (x2 - x1), y1 + t * (y2 - y1), z1 + t * (z2 - z1))
    return intersection


valid_image_corners_2d = []
backprojections = []

# Process each camera geometry
valid = []
for i in range(N):
    camera_plane = geometries[2 * i + 1].vertices
    camera_center = geometries[2 * (i + 1)].vertices[6]
    image_center_3d = np.average(camera_plane, axis=0)
    plane_orientation = np.dot(plane_equation[0:3], image_center_3d - camera_center)
    if plane_orientation > 0:
        plane_equation = -1 * plane_equation

    valid_image_corners_3d = np.copy(camera_plane)
    corners_2d = np.array(
        [
            [0, input_height],
            [input_width, input_height],
            [input_width, 0],
            [0, 0],
        ]
    )
    # Adjust corners 0 and 1 if pointing away from ceiling
    epsilon = 0.1
    A = camera_plane[0]
    C = camera_center
    n = plane_equation[:3]
    I = plane_line_intersection(plane_equation, C, A)
    z = np.dot(A - C, I - C)
    if z < 0:
        B = camera_plane[3]
        CA = A - C
        AB = B - A
        u = -(epsilon * np.linalg.norm(CA) + np.dot(n, CA)) / np.dot(AB, n)

        valid_image_corners_3d[0] = (1 - u) * A + u * B
        corners_2d[0] = [0, (1 - u) * input_height]
    A = camera_plane[1]
    I = plane_line_intersection(plane_equation, C, A)
    z = np.dot(A - C, I - C)
    if z < 0:
        B = camera_plane[2]
        CA = A - C
        AB = B - A
        u = -(epsilon * np.linalg.norm(CA) + np.dot(n, CA)) / np.dot(AB, n)

        valid_image_corners_3d[1] = (1 - u) * A + u * B
        corners_2d[1] = [input_width, (1 - u) * input_height]
    # valid_image_corners_2d stores the 2d corners (in pixels) of the image
    # that give rays pointing towards the ceiling
    valid_image_corners_2d.append(corners_2d)
    # backprojections stores these corners after backprojection on the ceiling:
    # after backprojection, we get 3d points
    valid.append(valid_image_corners_3d)
    backprojection = []
    for j in range(4):
        I = plane_line_intersection(plane_equation, C, valid_image_corners_3d[j])
        backprojection.append(I)
    backprojections.append(backprojection)

print("3D coordinates of the intersections : ", valid_image_corners_3d)


3D coordinates of the intersections :  [[-2.31943355  1.72613535  1.439666  ]
 [-2.38415879  1.82273376  1.67985162]
 [-2.265975    1.92152667  1.66438305]
 [-2.2103622   1.81731093  1.42539012]]


## Visualisation of the intersections for a chosen image

In [27]:
trace_points = []
trace_meshes = []

colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta']

camera_id = 0
color_counter = 0

# Loop through each geometry in the mesh (either a point cloud or a mesh)
for name, geometry in mesh.geometry.items():
    # If the geometry is a point cloud (the ceiling)
    if isinstance(geometry, trimesh.PointCloud):
        # Limit to 1500 points randomly sampled to avoid memory issues in VSCode
        num_points = min(1500, len(geometry.vertices))
        sampled_points = np.random.choice(len(geometry.vertices), num_points, replace=False)
        points = geometry.vertices[sampled_points]
        # Add the point cloud to the list for visualization
        trace_points.append(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(color=colors[color_counter % len(colors)], size=2),
            name=f"PointCloud {name}"
        ))
        color_counter += 1

    # If the geometry is a mesh (used for camera models)
    elif isinstance(geometry, trimesh.Trimesh):
        for face in geometry.faces:
            vertices = geometry.vertices[face]
            # Ensure that the face has 3 unique vertices
            if len(np.unique(vertices, axis=0)) == 3:
                trace_meshes.append(go.Mesh3d(
                    x=vertices[:, 0],
                    y=vertices[:, 1],
                    z=vertices[:, 2],
                    color=colors[camera_id % len(colors)],
                    opacity=1,
                    name=f"Camera #{camera_id}"
                ))

        if len(geometry.faces) == 48:
            camera_id += 1

# ===== Ceiling plane calculation =====

# Collect valid backprojection intersection points (not None)
valid_bps = []
for i in range(len(backprojections)):
    if i == 2:
        bp_list = backprojections[i]
        if isinstance(bp_list, list):
            for bp in bp_list:
                if bp is not None:
                    valid_bps.append(bp)

# Combine valid backprojection points with top_points if available
if valid_bps:
    valid_bps = np.array(valid_bps)
    combined_points = np.vstack([top_points, valid_bps])
else:
    combined_points = top_points

# Create a grid to visualize the plane
xlim = [combined_points[:, 0].min(), combined_points[:, 0].max()]
ylim = [combined_points[:, 1].min(), combined_points[:, 1].max()]
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 10), np.linspace(ylim[0], ylim[1], 10))
a, b, c, d = plane_equation
zz = (-a * xx - b * yy - d) / c

# Add the ceiling plane as a surface
trace_meshes.append(go.Surface(
    x=xx, y=yy, z=zz, opacity=0.2, colorscale='Oranges', showscale=False,
    name='Ceiling plane'
))

# ===== Compute bounding box to ensure uniform scale across axes =====

# Repeat valid point gathering to compute full bounds
valid_bps = []
for i in range(len(backprojections)):
    if i == 2:
        bp_list = backprojections[i]
        if isinstance(bp_list, list):
            for bp in bp_list:
                if bp is not None:
                    valid_bps.append(bp)

# Combine again with either all valid backprojections or just top_points
if valid_bps:
    valid_bps = np.array(valid_bps)
    combined_points = np.vstack([points, valid_bps])
else:
    combined_points = top_points

# Calculate min/max for each axis
xlim = [combined_points[:, 0].min(), combined_points[:, 0].max()]
ylim = [combined_points[:, 1].min(), combined_points[:, 1].max()]
zlim = [combined_points[:, 2].min(), combined_points[:, 2].max()]

dx = xlim[1] - xlim[0]
dy = ylim[1] - ylim[0]
dz = zlim[1] - zlim[0]

# Adjust axis ranges so all axes have the same scale
if dx >= dy and dx >= dz:
    xmin, xmax = (xlim[0] + xlim[1]) / 2 - dx / 2, (xlim[0] + xlim[1]) / 2 + dx / 2
    ymin, ymax = (ylim[0] + ylim[1]) / 2 - dx / 2, (ylim[0] + ylim[1]) / 2 + dx / 2
    zmin, zmax = (zlim[0] + zlim[1]) / 2 - dx / 2, (zlim[0] + zlim[1]) / 2 + dx / 2
elif dy >= dx and dy >= dz:
    xmin, xmax = (xlim[0] + xlim[1]) / 2 - dy / 2, (xlim[0] + xlim[1]) / 2 + dy / 2
    ymin, ymax = (ylim[0] + ylim[1]) / 2 - dy / 2, (ylim[0] + ylim[1]) / 2 + dy / 2
    zmin, zmax = (zlim[0] + zlim[1]) / 2 - dy / 2, (zlim[0] + zlim[1]) / 2 + dy / 2
else:
    xmin, xmax = (xlim[0] + xlim[1]) / 2 - dz / 2, (xlim[0] + xlim[1]) / 2 + dz / 2
    ymin, ymax = (ylim[0] + ylim[1]) / 2 - dz / 2, (ylim[0] + ylim[1]) / 2 + dz / 2
    zmin, zmax = (zlim[0] + zlim[1]) / 2 - dz / 2, (zlim[0] + zlim[1]) / 2 + dz / 2

# ===== Create 3D layout for visualization =====
layout = go.Layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(range=[xmin, xmax]),
        yaxis=dict(range=[ymin, ymax]),
        zaxis=dict(range=[zmin, zmax])
    ),
    title='Cameras, point cloud, and estimated ceiling plane',
    showlegend=True
)

# ===== Add projection rays and intersection points =====
for i in range(len(backprojections)):
    if i == 2: # Chosen camera 
        if isinstance(backprojections[i], list):
            cam_center = geometries[2 * (i + 1)].vertices[6]  # Camera optical center
            for j, bp in enumerate(backprojections[i]):
                if bp is None:
                    continue
                # Draw a line from camera center to intersection point
                trace_meshes.append(go.Scatter3d(
                    x=[cam_center[0], bp[0]],
                    y=[cam_center[1], bp[1]],
                    z=[cam_center[2], bp[2]],
                    mode='lines',
                    line=dict(color=colors[i % len(colors)], width=3),
                    name=f"Ray {i}-{j}",
                    showlegend=False
                ))
                # Draw the intersection point as a black dot
                trace_meshes.append(go.Scatter3d(
                    x=[bp[0]],
                    y=[bp[1]],
                    z=[bp[2]],
                    mode='markers',
                    marker=dict(color='black', size=4, symbol='circle'),
                    name=f"Intersection {i}-{j}",
                    showlegend=False
                ))

# ===== Show the final figure =====
fig = go.Figure(data=trace_points + trace_meshes, layout=layout)
fig.show()


# Camera representation visualization in Mat3er

In [28]:
i = 2  # Camera chosen to display

# Retrieve the camera center
cam_center = geometries[2*(i+1)].vertices[6]

# The camera's image plane
camera_plane = geometries[2*i+1].vertices

# valid_image_corners_3d corresponding to i
valid_points = valid[2]  # List of the 4 validated 3D points

# Points A, B, C, D (corners of the image plane)
A = camera_plane[3]
B = camera_plane[2]
C = camera_plane[1]
D = camera_plane[0]

# Validated image points C' and D' (valid_image_corners_3d[1] and [0])
C_prime = valid_points[1]
D_prime = valid_points[0]
print(valid_image_corners_3d[i])
trace_points = []
trace_meshes = []

colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta']

# 1) Plot the camera center (vertex) as a point
trace_points.append(go.Scatter3d(
    x=[cam_center[0]], y=[cam_center[1]], z=[cam_center[2]],
    mode='markers+text',
    marker=dict(size=6, color='black'),
    text="O",
    name='Camera center'
))

# 2) Plot the 4 corners of the camera plane (A, B, C, D) as points
plane_points = np.array([A, B, C, D])
trace_points.append(go.Scatter3d(
    x=plane_points[:, 0], y=plane_points[:, 1], z=plane_points[:, 2],
    mode='markers+text',
    marker=dict(size=5, color='blue'),
    text=['A', 'B', 'C', 'D'],
    textposition='top center',
    name='Camera plane corners'
))

# 3) Plot the 2 validated points C' and D' in red
valid_pts = np.array([C_prime, D_prime])
trace_points.append(go.Scatter3d(
    x=valid_pts[:, 0], y=valid_pts[:, 1], z=valid_pts[:, 2],
    mode='markers+text',
    marker=dict(size=5, color='red'),
    text=["C'", "D'"],
    textposition='top center',
    name='Valid corners'
))

# 4) Draw 4 lines from the camera center to A, B, C', and D', extending them slightly

def extend_line(p1, p2, scale=1):
    """ Extend the line from p1 to p2 by 'scale' times the vector length """
    direction = p2 - p1
    return p1 + scale * direction

lines = [
    (cam_center, A),
    (cam_center, B),
    (cam_center, C_prime),
    (cam_center, D_prime)
]

for idx, (start, end) in enumerate(lines):
    extended_end = extend_line(np.array(start), np.array(end))
    trace_meshes.append(go.Scatter3d(
        x=[start[0], extended_end[0]],
        y=[start[1], extended_end[1]],
        z=[start[2], extended_end[2]],
        mode='lines',
        line=dict(color='green' if idx < 2 else 'red', width=4),
        name=f'Ray {idx+1}'
    ))

# Set up layout (adapted to fit the points in view)
all_points = np.vstack([plane_points, valid_pts, cam_center.reshape(1,3)])
xlim = [all_points[:, 0].min(), all_points[:, 0].max()]
ylim = [all_points[:, 1].min(), all_points[:, 1].max()]
zlim = [all_points[:, 2].min(), all_points[:, 2].max()]
dx = xlim[1] - xlim[0]
dy = ylim[1] - ylim[0]
dz = zlim[1] - zlim[0]
max_range = max(dx, dy, dz)

center_x = (xlim[0] + xlim[1]) / 2
center_y = (ylim[0] + ylim[1]) / 2
center_z = (zlim[0] + zlim[1]) / 2

layout = go.Layout(
    scene=dict(
        xaxis=dict(range=[center_x - max_range/2, center_x + max_range/2], title='X'),
        yaxis=dict(range=[center_y - max_range/2, center_y + max_range/2], title='Y'),
        zaxis=dict(range=[center_z - max_range/2, center_z + max_range/2], title='Z')
    ),
    title=f"Camera i={i}: center, plane, validated points and rays"
)

fig = go.Figure(data=trace_points + trace_meshes, layout=layout)
fig.show()


[-2.265975    1.92152667  1.66438305]
